# 📦 Project: The Shipping Crisis (Saudi Retail Case Study)
## 📋 The Scenario
You have been hired by a major Saudi-based e-commerce platform. Management has noticed a significant decline in customer satisfaction scores. Initial reports suggest that a massive number of packages are not reaching customers on time.

**The Management's Question:**
*"We are losing money and customers. We need to know: Is this a logistics problem (Mode of Shipment), a product problem (Weight/Cost), or a strategy problem (Discounts/Priority)? Find the root cause and build a system to save our customers."*

## Phase 1: Data Understanding
In this phase, explore the `Saudi_Retail_Logistics_Data.csv` to find out **why** delays are happening.

* **Step 1.1:** Load the dataset.
* **Step 1.2:** Perform statistical profiling (`.describe()`, `.info()`).
* **Step 1.3:** Create visualizations (Histograms/Scatter) to find the problem.

| Feature Name         | Type       | Description                                                                 | Values / Examples                               |
| :------------------- | :--------- | :-------------------------------------------------------------------------- | :--------------------------------------------- |
| Tracking_ID          | String     | A unique, encrypted alphanumeric identifier for each shipment.              | E3B0C44298FC, 1C2D3E4F5G6H                     |
| Warehouse_Zone       | Categorical| The regional distribution hub handling the order.                           | North_Hub, South_Hub, East_Hub, West_Hub, Central_Port |
| Mode_of_Shipment     | Categorical| The logistics method used to transport the product.                         | Premium_Air, Standard_Sea, Ground_Freight      |
| Customer_Inquiries   | Integer    | Number of times the customer contacted support regarding this order.        | 2, 3, 4, etc.                                  |
| Customer_Score       | Integer    | The rating provided by the customer to the store before shipment (1 is lowest, 5 is highest).            | 1, 2, 3, 4, 5                                  |
| Product_Cost_SAR     | Float      | The value of the item in Saudi Riyals (SAR).                                | 450.00, 1125.50, etc.                          |
| Previous_Orders      | Integer    | Number of successful orders this customer has placed previously.            | 0, 1, 2, 3, etc.                               |
| Product_Priority     | Categorical| The priority level assigned to the product for dispatch.                    | low, medium, high                              |
| Gender               | Categorical| The gender of the customer.                                                 | F, M                                           |
| Discount_offered     | Integer    | The discount amount applied to this specific order.                         | 1, 10, 65, etc.                                |
| Product_Weight_g     | Integer    | The weight of the package in grams.                                         | 1200, 4500, etc.                               |
| is_delayed           | Binary     | Target Variable: Indicates if the shipment arrived past the deadline.       | 1 (Delayed), 0 (On Time)                       |

In [1]:
from pathlib import Path
import pandas as pd

local_file = Path('Saudi_Retail_Logistics_Data.csv')

if not local_file.exists():
    print("Downloading data...")
    df = pd.read_csv('https://drive.google.com/uc?export=download&id=1vayY_XKPwe5lOaq0VNia22O4DkDVXmzv')
    df.to_csv(local_file, index=False)
else:
    print("Loading data from cache...")
    df = pd.read_csv(local_file)
df.head()

Loading data from cache...


,Tracking_ID,Warehouse_Zone,Mode_of_Shipment,Customer_Inquiries,Customer_Score,Product_Cost_SAR,Previous_Orders,Product_Priority,Gender,Discount_offered,Product_Weight_g,is_delayed
0,D66B138C54AB,West_Hub,Ground_Freight,4,1,978.75,2,medium,F,9,4158,0
1,8FF9538E65E6,Central_Port,Standard_Sea,3,3,607.50,2,low,F,26,1659,1
2,D098284FA376,South_Hub,Standard_Sea,3,2,828.75,2,medium,M,9,4466,0
3,9202E9FDFF40,West_Hub,Standard_Sea,3,5,588.75,4,medium,F,2,4640,0
4,16B30490A644,North_Hub,Premium_Air,2,5,1020.00,2,low,M,24,3638,1


In [2]:
import plotly.express as px


num_features_screening = [
    'Customer_Inquiries', 'Customer_Score', 'Product_Cost_SAR', 
    'Previous_Orders', 'Discount_offered', 'Product_Weight_g'
]


for col in num_features_screening:
    fig = px.histogram(
        df, 
        x=col, 
        color="is_delayed",           
        title=f"Distribution of {col} vs. Shipping Delays",
        marginal="box",               
        template="plotly_white",
        barmode="overlay",            
        color_discrete_sequence=['#636EFA', '#EF553B'], 
        labels={'is_delayed': 'Status (1=Delayed)'}
    )
    
    fig.update_layout(bargap=0.1)
    fig.show()

Why are shipments delayed?

- Discounts over 10%: When the discount is high, people buy more. This creates too many orders at once, and the warehouse cannot keep up.

- Products with (2000g – 4000g) weight: Products in this weight range are always delayed.

- Small Products with (1000g – 2000g) weight: These items have a 50% chance of being delayed.

Conclusion: Our shipping company likely struggles to handle these specific weights, and our system gets overwhelmed during big sales.

## Phase 2: Feature Engineering & Baseline
Prepare the data for Machine Learning.

In [8]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report


from pycaret import *

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

features = [
    'Warehouse_Zone', 'Mode_of_Shipment', 'Customer_Inquiries',
    'Customer_Score', 'Product_Cost_SAR', 'Previous_Orders',
    'Product_Priority', 'Gender', 'Discount_offered', 'Product_Weight_g'
]


X = df[features]
y = df['is_delayed']


X_encoded = pd.get_dummies(X, columns=['Warehouse_Zone', 'Mode_of_Shipment', 'Product_Priority', 'Gender'], drop_first=True)


scaler = StandardScaler()
X_final = pd.DataFrame(scaler.fit_transform(X_encoded), columns=X_encoded.columns)


X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

print(f"Data is ready! Training shape: {X_train.shape}")

Data is ready! Training shape: (7039, 15)


## Phase 3: Training, Evaluation & Strategy
Build your solution and define your business interventions.

* **Step 3.1:** Train a Classifier.
* **Step 3.2:** Generate a Classification Report and Confusion Matrix.
* **Step 3.3:** **CRITICAL:** List 3 business solution will save our customers based on your model (you can use a 5% Coupon).

In [13]:
from autogluon.tabular import TabularPredictor


label = 'is_delayed'
eval_metric = 'recall' 

predictor = TabularPredictor(
    label=label, 
    eval_metric=eval_metric
).fit(
    train_data=df, 
    presets='best_quality',
    time_limit=600  
)


leaderboard = predictor.leaderboard(df)
print(leaderboard)

No path specified. Models will be saved in: "AutogluonModels\ag-20260103_174013"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26200
CPU Count:          8
Pytorch Version:    2.7.1+cpu
CUDA Version:       CUDA is not available
Memory Avail:       1.86 GB / 15.77 GB (11.8%)
Disk Space Avail:   21.60 GB / 952.53 GB (2.3%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	Thi

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: c189a8d650cf16949a2d3749b524d801aad1373601000000 Worker ID: 15a399db404b2956d0835baebadc9d7462ae6bdf0700c46bf8ea409b Node ID: e73b7e55eb7c788a31effd82c5f7093eeacdf6060342e7ec1fba747f Worker IP address: 127.0.0.1 Worker port: 3946 Worker PID: 54832 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 10054. An existing connection was forcibly closed by the remote host. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


(_dystack pid=46440) Running DyStack sub-fit ...
(_ray_fit pid=2384) 	Warning: Model has no time left to train, skipping model... (Time Left = -12.6s) [repeated 7x across cluster]
(_dystack pid=46440) Beginning AutoGluon training ... Time limit = 132s
(_dystack pid=46440) AutoGluon will save models to "c:\Users\Komail AL-Basri\bootcoamp\week3-ml-baseline-system\notebooks\AutogluonModels\ag-20260103_174013\ds_sub_fit\sub_fit_ho"
(_dystack pid=46440) Train Data Rows:    7821
(_dystack pid=46440) Train Data Columns: 11
(_dystack pid=46440) Label Column:       is_delayed
(_dystack pid=46440) Problem Type:       binary
(_dystack pid=46440) Preprocessing data ...
(_dystack pid=46440) Selected class <--> label mapping:  class 1 = 1, class 0 = 0
(_dystack pid=46440) Using Feature Generators to preprocess the data ...
(_dystack pid=46440) Fitting AutoMLPipelineFeatureGenerator...
(_dystack pid=46440) 	Available Memory:                    4626.19 MB
(_dystack pid=46440) 	Train Data (Original)  M

                      model  score_test  score_val eval_metric  \
0           LightGBM_BAG_L1    1.000000   1.000000      recall   
1   RandomForestGini_BAG_L1    1.000000   0.620952      recall   
2   RandomForestEntr_BAG_L1    1.000000   0.621905      recall   
3     ExtraTreesEntr_BAG_L1    1.000000   0.637524      recall   
4     ExtraTreesGini_BAG_L1    1.000000   0.639048      recall   
5         LightGBMXT_BAG_L1    1.000000   1.000000      recall   
6       WeightedEnsemble_L2    1.000000   1.000000      recall   
7       WeightedEnsemble_L3    1.000000   1.000000      recall   
8           LightGBM_BAG_L2    1.000000   1.000000      recall   
9         LightGBMXT_BAG_L2    1.000000   1.000000      recall   
10          CatBoost_BAG_L2    0.901905   0.518476      recall   
11  RandomForestEntr_BAG_L2    0.842857   0.607048      recall   
12  RandomForestGini_BAG_L2    0.802095   0.607238      recall   
13          CatBoost_BAG_L1    0.694095   0.622857      recall   
14   Neura

### 📝 Business Strategy Recommendations
1. 
2. 
3. 

## Phase 4: The Business Value Calculator
Calculate the SAR impact of your model using the following values:
* **+ SAR 75.00**: True Positive (Saved Customer)
* **- SAR 18.75**: False Positive (5% Wasted Coupon)
* **- SAR 187.50**: False Negative (Lost Lifetime Value)

In [16]:

#tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn = 32
fp =32
fn = 1033
tp = 1543

tp_value = 75.00    
fp_cost = -18.75    
fn_cost = -187.50   

total_impact = (tp * tp_value) + (fp * fp_cost) + (fn * fn_cost)


print(f"True Positives  (Saved):  {tp} x SAR 75.00  = SAR {tp * tp_value:,.2f}")
print(f"False Positives (Waste):  {fp} x SAR 18.75  = SAR {fp * fp_cost:,.2f}")
print(f"False Negatives (Loss):   {fn} x SAR 187.50 = SAR {fn * fn_cost:,.2f}")
print(f"----------------------------------------------")
print(f"TOTAL  VALUE: SAR {total_impact:,.2f}")

True Positives  (Saved):  1543 x SAR 75.00  = SAR 115,725.00
False Positives (Waste):  32 x SAR 18.75  = SAR -600.00
False Negatives (Loss):   1033 x SAR 187.50 = SAR -193,687.50
----------------------------------------------
TOTAL  VALUE: SAR -78,562.50


In [13]:
print(df.count())

Tracking_ID           8799
Warehouse_Zone        8799
Mode_of_Shipment      8799
Customer_Inquiries    8799
Customer_Score        8799
Product_Cost_SAR      8799
Previous_Orders       8799
Product_Priority      8799
Gender                8799
Discount_offered      8799
Product_Weight_g      8799
is_delayed            8799
dtype: int64
